In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.layers.recurrent import LSTM
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

In [2]:
Raw_data=pd.read_csv('D:\Code\GoldMLmodel\merged_df.csv')
data = Raw_data.copy()

C:\Users\ea1\AppData\Local\Temp\ipykernel_13852\905991724.py:1: DtypeWarning: Columns (1,41,45,46,47,49,73,78,82,83,84,85,86,94,95,96,97,98,113,116,118,122,123,124,125,126,135,137,141,143,145,147,149,151,153,156,157,158,161,162,163,164,165,166,168,171,172,173,174,180,182,190,192,201,202,203,204,205,234,237,238,239,240,241,242,244,267,269,272,274,278,279,280,281,282,288,290,305,307,316,317,318,319,320,343,345,346,347,349,350,351,352,353,384,393,394,395,396,397,437,438,439,440,441,443,447,449,475,476,477,478,479,513,514,515,516,517,528,545,547,551,552,553,555,569,571,577,579,588,589,590,591,592,626,629,630,631,632,633,634,636,639,640,641,642,661,670,671,672,673,674,714,715,716,717,718,727,728,730,746,748,752,753,754,755,756,765,777,778,780,789,790,791,792,793,823,827,828,829,830,831,833,836,837,838,839,846,848,850,858,867,868,869,870,871,909,910,911,912,913,943,945,946,947,948,949,950,951,952,953,955,958,959,960,961,968,970,978,980,982,991,992,993,994,995,1006,1028,1029,1030,1031,1032,10

In [3]:
data['DateTime'] = pd.to_datetime(data['DateTime'])
data = data.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name != 'DateTime' else x ).fillna(0)
data.set_index(['DateTime'],inplace=True)
data

,Weekly_EoPDfrencyricesUSD_,Weekly_EoPDfrencyricesEUR_,Weekly_EoPDfrencyricesJPY_,Weekly_EoPDfrencyricesGBP_,Weekly_EoPDfrencyricesCAD_,Weekly_EoPDfrencyricesCHF_,Weekly_EoPDfrencyricesINR_,Weekly_EoPDfrencyricesCNY_,Weekly_EoPDfrencyricesTRY_,Weekly_EoPDfrencyricesSAR_,...,Consumer_Other & stock change_gold_demand_prices_supply4,Consumer_World total_gold_demand_prices_supply4,+gold_cost_supply6US$/oz_,+gold_cost_supply6€/oz_,+gold_cost_supply6£/oz_,#NAME?,+gold_cost_supply6¥/g_,+gold_cost_supply6Rs/10g_,#NAME?.1,#NAME?.2
DateTime,,,,,,,,,,,,,,,,,,,,,
1905-07-06 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
1905-07-07 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
1905-07-08 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
1905-07-09 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
1905-07-10 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,224.635185,3282.111382,1889.92,1761.01,1554.55,56218.75352,8042.302635,49943.79411,415.694697,1147.583712
2023-07-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,224.635185,3282.111382,1975.93,1815.13,1577.58,57085.85850,8739.014259,52191.57008,446.106387,1339.678814
2023-09-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,224.635185,3282.111382,1928.48,1772.20,1523.05,54768.11291,8961.098912,51259.75212,449.144630,1662.144132


In [4]:
columns_to_drop = []

# Iterate through columns and drop those with more than 14000 null values
for column in data.columns:
    null_count = data[column].isna().sum()
    if null_count > 14500:
        print(f"Column '{column}' has {null_count} null values and will be dropped.")
        columns_to_drop.append(column)

# Drop & display columns with more than 14000 null values
df_cleaned = data.drop(columns=columns_to_drop)
print(df_cleaned)

                     Weekly_EoPDfrencyricesUSD_  Weekly_EoPDfrencyricesEUR_  \
DateTime                                                                      
1905-07-06 00:00:00                         0.0                         0.0   
1905-07-07 00:00:00                         0.0                         0.0   
1905-07-08 00:00:00                         0.0                         0.0   
1905-07-09 00:00:00                         0.0                         0.0   
1905-07-10 00:00:00                         0.0                         0.0   
...                                         ...                         ...   
2023-04-01 00:00:00                         0.0                         0.0   
2023-07-01 00:00:00                         0.0                         0.0   
2023-09-01 00:00:00                         0.0                         0.0   
2023-12-01 00:00:00                         0.0                         0.0   
2023-12-29 00:00:00                         0.0     

In [5]:
rows_to_drop = []

# Iterate through rows and drop those with more than 14000 null values
for index, row in df_cleaned.iterrows():
    null_count = row.isna().sum()
    if null_count > 11000:
        print(f"Row at index {index} has {null_count} null values and will be dropped.")
        rows_to_drop.append(index)

# Drop & display rows with more than 14000 null values
ddf_cleaned = df_cleaned.drop(index=rows_to_drop)
print(ddf_cleaned)

                     Weekly_EoPDfrencyricesUSD_  Weekly_EoPDfrencyricesEUR_  \
DateTime                                                                      
1905-07-06 00:00:00                         0.0                         0.0   
1905-07-07 00:00:00                         0.0                         0.0   
1905-07-08 00:00:00                         0.0                         0.0   
1905-07-09 00:00:00                         0.0                         0.0   
1905-07-10 00:00:00                         0.0                         0.0   
...                                         ...                         ...   
2023-04-01 00:00:00                         0.0                         0.0   
2023-07-01 00:00:00                         0.0                         0.0   
2023-09-01 00:00:00                         0.0                         0.0   
2023-12-01 00:00:00                         0.0                         0.0   
2023-12-29 00:00:00                         0.0     

In [6]:
ddata = ddf_cleaned.replace(np.nan,0)
ddata.reset_index(drop=True, inplace=True)
ddata = ddata.interpolate(method='polynomial', order=2, axis=0)
ddata

,Weekly_EoPDfrencyricesUSD_,Weekly_EoPDfrencyricesEUR_,Weekly_EoPDfrencyricesJPY_,Weekly_EoPDfrencyricesGBP_,Weekly_EoPDfrencyricesCAD_,Weekly_EoPDfrencyricesCHF_,Weekly_EoPDfrencyricesINR_,Weekly_EoPDfrencyricesCNY_,Weekly_EoPDfrencyricesTRY_,Weekly_EoPDfrencyricesSAR_,...,Consumer_Other & stock change_gold_demand_prices_supply4,Consumer_World total_gold_demand_prices_supply4,+gold_cost_supply6US$/oz_,+gold_cost_supply6€/oz_,+gold_cost_supply6£/oz_,#NAME?,+gold_cost_supply6¥/g_,+gold_cost_supply6Rs/10g_,#NAME?.1,#NAME?.2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,224.635185,3282.111382,1889.92,1761.01,1554.55,56218.75352,8042.302635,49943.79411,415.694697,1147.583712
14522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,224.635185,3282.111382,1975.93,1815.13,1577.58,57085.85850,8739.014259,52191.57008,446.106387,1339.678814
14523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,224.635185,3282.111382,1928.48,1772.20,1523.05,54768.11291,8961.098912,51259.75212,449.144630,1662.144132
14524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,224.635185,3282.111382,1971.49,1834.57,1590.09,56317.13473,9389.746813,52761.22301,458.725545,1806.651985


In [11]:
x=ddata.drop(columns= list(ddata.columns[19:38]))
y=ddata.iloc[:,19]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)
x.shape

(14526, 11719)

In [8]:
model = RandomForestRegressor()
model.fit(x_train, y_train)

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
x, y = make_regression(n_features=4, n_informative=2,
                       random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x, y)
RandomForestRegressor(...)
print(regr.predict([[0, 0, 0, 0]]))

score = model.score(x_test, y_test)
print(score)
y_pred = model.predict(x_test)
y_pred[:100]

[-8.32987858]
0.9997893598921201


array([ 381.1705,    0.    ,  336.838 ,    0.    , 1539.64  , 1216.0635,
       1930.847 ,  903.345 ,  637.303 ,    0.    , 1656.919 ,  281.2535,
       1674.425 , 1735.6845,  287.1715,    0.    , 1335.8695,  630.569 ,
        371.076 , 1754.964 ,    0.    ,  341.4085,  357.916 ,    0.    ,
       1413.933 ,    0.    ,  361.894 ,  375.695 ,  485.386 ,  671.535 ,
        366.0415,  240.681 ,    0.    ,  340.0935,  298.886 ,    0.    ,
       1501.3865,  932.355 , 1190.8795,  446.393 ,    0.    ,  298.827 ,
       1194.1035,  457.715 ,  994.5075,  303.4225, 1326.627 ,    0.    ,
          0.    ,  308.308 ,  381.2787,    0.    , 1775.733 ,  956.5075,
        419.747 ,  459.833 ,  388.554 ,  527.733 ,  906.1875, 1687.9985,
        401.111 ,  346.426 ,    0.    , 1006.875 ,  421.133 ,  413.606 ,
       1157.9565,  358.25  ,    0.    ,  323.4885,    0.    ,  362.5535,
       1532.966 , 1276.732 ,  980.06  , 1282.2125,    0.    ,  327.559 ,
          0.    , 1931.486 , 1373.885 ,  366.009 , 

In [13]:
t= ddata['USD_DDffrentCurcyPrcs_'].shift(-1)
t.fillna(0, inplace=True)
x_train,x_test,t_train,t_test=train_test_split(x,t,test_size=0.25)
# t.shape

In [14]:
model = RandomForestRegressor().fit(x_train, t_train)
score = model.score(x_test, t_test)
print(score)
t_pred = model.predict(x_test)
t_pred

0.9961503298285862


array([  0.    , 396.5525, 294.653 , ..., 369.7545, 239.1145, 457.8185])

In [15]:
model = LinearRegression().fit(x_train, t_train)
# Make predictions on the test set
t_pred = model.predict(x_test)

# Evaluate the model
mse = mean_squared_error(t_test, t_pred)
mae = mean_absolute_error(t_test, t_pred)
r2 = r2_score(t_test, t_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

Mean Squared Error: 301731.99093928916
Mean Absolute Error: 16.573814009563236
R-squared: 0.0628484219162414


In [16]:
model = LinearRegression().fit(x_train, y_train)
# Make predictions on the test set
y_pred = model.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
score = model.score(x_test, y_test)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)
print("R-squared score:", score)

Mean Squared Error: 6369065551.321358
Mean Absolute Error: 2137.718805663941
R-squared: -19466.032101572968
R-squared score: -19466.032101572968


In [17]:
model = keras.Sequential()
model.add(Dense(units=256, input_shape=(11721,), activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1,activation='relu'))
model.compile(optimizer='adam', loss='mse')
model.fit(x_train, y_train, epochs=100,batch_size = 1000)
# model.evaluate(x_test,y_test)
y_pred = model.predict(x_test)
print(y_pred)
mse = mean_squared_error(y_test, y_pred)
print(mse)



Epoch 1/100

11/11 [==============================] - 3s 121ms/step - loss: 688481.1875
Epoch 2/100
11/11 [==============================] - 1s 137ms/step - loss: 688481.2500
Epoch 3/100
11/11 [==============================] - 1s 126ms/step - loss: 688481.1250
Epoch 4/100
11/11 [==============================] - 1s 127ms/step - loss: 688481.1875
Epoch 5/100
11/11 [==============================] - 1s 127ms/step - loss: 688481.1250
Epoch 6/100
11/11 [==============================] - 1s 123ms/step - loss: 688481.1875
Epoch 7/100
11/11 [==============================] - 1s 124ms/step - loss: 688481.1250
Epoch 8/100
11/11 [==============================] - 1s 125ms/step - loss: 688481.1250
Epoch 9/100
11/11 [==============================] - 2s 142ms/step - loss: 688481.1250
Epoch 10/100
11/11 [==============================] - 2s 166ms/step - loss: 688481.1875
Epoch 11/100
11/11 [==============================] - 2s 168ms/step - loss: 688481.2500
Epoch 12/100
11/11 [==================

In [18]:
from tensorflow.keras.optimizers import Adam
model = keras.Sequential()
model.add(Dense(units=256, input_shape=(11554,), activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1))
optimizer=Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse')
model.fit(x_train, t_train, epochs=100,batch_size = 1000)
# model.evaluate(x_test,t_test)

Epoch 1/100
11/11 [==============================] - 3s 148ms/step - loss: nan   
Epoch 2/100
11/11 [==============================] - 2s 141ms/step - loss: nan
Epoch 3/100
11/11 [==============================] - 1s 132ms/step - loss: nan
Epoch 4/100
11/11 [==============================] - 2s 136ms/step - loss: nan
Epoch 5/100
11/11 [==============================] - 2s 148ms/step - loss: nan
Epoch 6/100
11/11 [==============================] - 2s 158ms/step - loss: nan
Epoch 7/100
11/11 [==============================] - 2s 177ms/step - loss: nan
Epoch 8/100
11/11 [==============================] - 2s 171ms/step - loss: nan
Epoch 9/100
11/11 [==============================] - 2s 170ms/step - loss: nan
Epoch 10/100
11/11 [==============================] - 2s 181ms/step - loss: nan
Epoch 11/100
11/11 [==============================] - 2s 179ms/step - loss: nan
Epoch 12/100
11/11 [==============================] - 2s 166ms/step - loss: nan
Epoch 13/100
11/11 [==========================

In [19]:
model = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print(y_pred)

mse = mean_squared_error(y_test, y_pred)
print(mse)

[723.48061318 616.43613162 605.02010706 ... 607.53110518 612.33181668
 616.43613162]
331601.3909024998


In [20]:
model = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
model.fit(x_train, t_train)

y_pred = model.predict(x_test)
print(t_pred)

mse = mean_squared_error(t_test, t_pred)
print(mse)

[  5.27199832 397.2455367  292.12767316 ... 366.36744357 238.45144844
 459.77121544]
301731.99093928916


In [21]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(11720,)),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer with single neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
model.evaluate(x_test, y_test)

Epoch 1/100

273/273 [==============================] - 5s 14ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
273/273 [==============================] - 3s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
273/273 [==============================] - 3s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/100
273/273 [====

[nan, nan]

In [22]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(11720,)),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer with single neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(x_train, t_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
model.evaluate(x_test, t_test)

Epoch 1/100
273/273 [==============================] - 5s 14ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
273/273 [==============================] - 3s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
273/273 [==============================] - 3s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
273/273 [==============================] - 3s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
273/273 [==============================] - 3s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/100
273/273 [======

[nan, nan]

In [26]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
transformer = PolynomialFeatures(degree=2, include_bias=False)
transformer.fit(x)
x_ = transformer.transform(x)
x_ = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x)
model = LinearRegression().fit(x_, y)
r_sq = model.score(x_, y)
intercept, coefficients = model.intercept_, model.coef_
y_pred = model.predict(x_)

MemoryError: Unable to allocate 7.26 TiB for an array with shape (14526, 68685059) and data type float64

In [25]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0,
    loss='squared_error'
).fit(x_train, y_train)
mean_squared_error(y_test, est.predict(x_test))

329507.11290655576